### Creation Object-centric log out of triplges generated by trackintel

In [1]:
import sys
import os
import pm4py
import json
# Assuming you're in notebooks/, go one level up
sys.path.append(os.path.abspath(".."))

from pm4mobility.io.loader import *
from pm4mobility.analysis import flows, transport
from pm4mobility.log import ocel_log
#from mobility4py.log.event_log import create_event_log

#### 2. Read triplges from geojson file

In [2]:
df = load_triplegs_geojson("../data/triplegs.geojson")

In [10]:
df.head(10)

,id,trip_id,started_at,finished_at,length,speed,duration_minutes,mode,geometry
0,13,0,2024-02-02 08:47:14+00:00,2024-02-02 08:47:42+00:00,325.714568,11.632663,0.466667,motorized_mobility,"LINESTRING (-9.15809 38.71762, -9.16165 38.71669)"
1,14,0,2024-02-02 09:11:50+00:00,2024-02-02 09:12:27+00:00,325.714568,8.803096,0.616667,motorized_mobility,"LINESTRING (-9.16165 38.71669, -9.15809 38.71762)"
2,19,1,2024-02-03 12:05:23+00:00,2024-02-03 12:08:59+00:00,325.714568,1.507938,3.600000,slow_mobility,"LINESTRING (-9.15809 38.71762, -9.15809 38.717..."
3,22,2,2024-02-05 20:06:19+00:00,2024-02-05 20:10:53+00:00,325.714568,1.188739,4.566667,slow_mobility,"LINESTRING (-9.16165 38.71669, -9.15809 38.71762)"
4,24,3,2024-02-06 11:26:38+00:00,2024-02-06 11:30:12+00:00,325.714568,1.522031,3.566667,slow_mobility,"LINESTRING (-9.16165 38.71669, -9.15809 38.71762)"
5,37,4,2024-02-13 12:31:44+00:00,2024-02-13 12:45:48+00:00,325.714568,0.385918,14.066667,slow_mobility,"LINESTRING (-9.16165 38.71669, -9.15809 38.717..."
6,38,5,2024-02-14 11:12:05+00:00,2024-02-14 11:17:00+00:00,325.714568,1.104117,4.916667,slow_mobility,"LINESTRING (-9.15809 38.71762, -9.16165 38.71669)"
7,42,6,2024-02-16 07:29:43+00:00,2024-02-16 07:33:22+00:00,325.714568,1.487281,3.650000,slow_mobility,"LINESTRING (-9.15809 38.71762, -9.16165 38.71669)"
8,54,7,2024-02-25 12:19:13+00:00,2024-02-25 12:21:27+00:00,325.714568,2.430706,2.233333,slow_mobility,"LINESTRING (-9.16165 38.71669, -9.16165 38.716..."
9,55,8,2024-02-25 20:15:11+00:00,2024-02-25 20:16:21+00:00,325.714568,4.653065,1.166667,motorized_mobility,"LINESTRING (-9.15809 38.71762, -9.16165 38.71669)"


#### 3. Read districts file 

In [3]:
districts = load_districts_layer("../data/cont-aad-caop2017")

In [12]:
districts.head(10)

,Dicofre,Freguesia,Concelho,Distrito,TAA,AREA_EA_Ha,AREA_T_Ha,Des_Simpli,geometry
0,080106,Albufeira e Olhos de Água,ALBUFEIRA,FARO,ÁREA SECUNDÁRIA,0.09,4117.69,Albufeira e Olhos de Água,"POLYGON ((-8.28204 37.07355, -8.28205 37.07354..."
1,081504,Sagres,VILA DO BISPO,FARO,ÁREA PRINCIPAL,3431.83,3436.91,Sagres,"POLYGON ((-8.89529 37.04182, -8.89533 37.04175..."
2,080106,Albufeira e Olhos de Água,ALBUFEIRA,FARO,ÁREA SECUNDÁRIA,0.01,4117.69,Albufeira e Olhos de Água,"POLYGON ((-8.30727 37.07449, -8.30727 37.07449..."
3,080106,Albufeira e Olhos de Água,ALBUFEIRA,FARO,ÁREA SECUNDÁRIA,0.01,4117.69,Albufeira e Olhos de Água,"POLYGON ((-8.30764 37.07458, -8.30766 37.07458..."
4,080106,Albufeira e Olhos de Água,ALBUFEIRA,FARO,ÁREA SECUNDÁRIA,0.01,4117.69,Albufeira e Olhos de Água,"POLYGON ((-8.30777 37.07466, -8.30778 37.07466..."
5,080106,Albufeira e Olhos de Água,ALBUFEIRA,FARO,ÁREA SECUNDÁRIA,0.01,4117.69,Albufeira e Olhos de Água,"POLYGON ((-8.30741 37.07488, -8.30742 37.07487..."
6,080106,Albufeira e Olhos de Água,ALBUFEIRA,FARO,ÁREA SECUNDÁRIA,0.01,4117.69,Albufeira e Olhos de Água,"POLYGON ((-8.30824 37.07497, -8.30825 37.07497..."
7,080106,Albufeira e Olhos de Água,ALBUFEIRA,FARO,ÁREA SECUNDÁRIA,0.01,4117.69,Albufeira e Olhos de Água,"POLYGON ((-8.27701 37.07564, -8.27702 37.07564..."
8,080106,Albufeira e Olhos de Água,ALBUFEIRA,FARO,ÁREA SECUNDÁRIA,0.02,4117.69,Albufeira e Olhos de Água,"POLYGON ((-8.27663 37.07574, -8.27664 37.07574..."
9,080106,Albufeira e Olhos de Água,ALBUFEIRA,FARO,ÁREA SECUNDÁRIA,0.01,4117.69,Albufeira e Olhos de Água,"POLYGON ((-8.27658 37.07594, -8.27658 37.07594..."


#### 4. Create Origin-Destination Flows out of triplges

In [4]:
od_flows = flows.create_od_flows(df, districts)

In [32]:
od_flows.head(10)

,trip_id,origin,started_at,destination,finished_at,origin_area,dest_area,origin_antenna,dest_antenna,origin_mode,dest_mode,duration_minutes,speed,origin_length,dest_length,estimated_mode
18,16,POINT (-9.18948 38.77278),2024-02-05 20:35:19+00:00,POINT (-9.19849 38.76623),2024-02-05 20:41:48+00:00,LISBOA,ODIVELAS,Carnide,Pontinha e Famões,slow_mobility,slow_mobility,6.483333,2.745047,1067.823472,1067.823472,bicycle
24,22,POINT (-9.18769 38.78616),2024-02-16 18:37:51+00:00,POINT (-9.18948 38.77278),2024-02-16 18:39:55+00:00,ODIVELAS,LISBOA,Odivelas,Carnide,motorized_mobility,motorized_mobility,2.066667,12.061921,1495.678151,1495.678151,unknown
27,25,POINT (-9.19206 38.79794),2024-02-27 19:05:57+00:00,POINT (-9.20114 38.80444),2024-02-27 19:10:39+00:00,ODIVELAS,ODIVELAS,Odivelas,Ramada e Caneças,slow_mobility,slow_mobility,4.700000,3.789262,1068.571802,1068.571802,bicycle
34,32,POINT (-9.19206 38.79794),2024-03-12 18:09:11+00:00,POINT (-9.18744 38.80551),2024-03-12 18:11:11+00:00,ODIVELAS,ODIVELAS,Odivelas,Ramada e Caneças,motorized_mobility,motorized_mobility,2.000000,7.763965,931.675799,931.675799,unknown
35,33,POINT (-9.12486 38.77111),2024-02-04 19:52:29+00:00,POINT (-9.1361 38.75947),2024-02-04 19:53:23+00:00,LISBOA,LISBOA,Olivais,Alvalade,fast_mobility,fast_mobility,0.900000,30.014186,1620.766057,1620.766057,long_distance_train
38,36,POINT (-9.14247 38.75119),2024-02-23 21:53:47+00:00,POINT (-9.12486 38.77111),2024-02-23 21:58:00+00:00,LISBOA,LISBOA,Alvalade,Olivais,motorized_mobility,motorized_mobility,4.216667,10.647495,2693.816128,2693.816128,unknown
39,37,POINT (-9.12486 38.77111),2024-02-25 17:07:58+00:00,POINT (-9.1361 38.75947),2024-02-25 17:10:14+00:00,LISBOA,LISBOA,Olivais,Alvalade,motorized_mobility,motorized_mobility,2.266667,11.917397,1620.766057,1620.766057,unknown
40,38,POINT (-9.12486 38.77111),2024-03-01 13:53:19+00:00,POINT (-9.1361 38.75947),2024-03-01 13:56:09+00:00,LISBOA,LISBOA,Olivais,Alvalade,motorized_mobility,motorized_mobility,2.833333,9.533918,1620.766057,1620.766057,unknown
42,39,POINT (-9.1361 38.75947),2024-03-05 19:46:06+00:00,POINT (-9.12486 38.77111),2024-03-05 19:48:24+00:00,LISBOA,LISBOA,Alvalade,Olivais,motorized_mobility,motorized_mobility,2.300000,11.744682,1620.766057,1620.766057,unknown
46,42,POINT (-9.19773 38.74461),2024-02-05 17:37:38+00:00,POINT (-9.20687 38.74136),2024-02-05 17:41:03+00:00,LISBOA,AMADORA,Benfica,Águas Livres,motorized_mobility,motorized_mobility,3.416667,4.250192,871.289349,871.289349,bicycle


#### 5. Estimate transportation mode

In [5]:
od_flows['estimated_mode'] = transport.estimate_transport_mode(od_flows['origin_length'], od_flows['duration_minutes'], od_flows['speed'])

#### 6. Filtering flows only to municipality of Oeiras

In [19]:
oeiras = od_flows.loc[(od_flows['origin_area'] == 'OEIRAS') #| (od_flows['dest_area'] == 'OEIRAS')
                      ]

In [20]:
oeiras_with_modes = oeiras.loc[(oeiras['duration_minutes'] > 3) & (oeiras['estimated_mode'] != 'unknown')]

#### 6. Create object-centric event log out of oeiras origin destination flows

In [21]:
ocel_df = ocel_log.create_ocel_from_dataframe(oeiras)

In [22]:
ocel_df_with_mode = ocel_log.create_ocel_from_dataframe(oeiras_with_modes, origin_mode_col='estimated_mode',
    dest_mode_col='estimated_mode')

In [23]:
with open('../data/ocel_log_oeiras.json', 'w') as f:
    json.dump(ocel_df, f, indent=2)

In [24]:
with open('../data/ocel_log_oeiras_with_mode.json', 'w') as f:
    json.dump(ocel_df_with_mode, f, indent=2)

#### 7. Read saved JSON logs with PM4PY

In [25]:
ocel_df_oeiras =  pm4py.read_ocel2_json("../data/ocel_log_oeiras.json")
ocel_df_oeiras_with_mode = pm4py.read_ocel2_json("../data/ocel_log_oeiras_with_mode.json")

In [29]:
ocel_df_oeiras.get_extended_table()

,ocel:eid,ocel:timestamp,ocel:activity,ocel:type:motorized_mobility,ocel:type:fast_mobility,ocel:type:slow_mobility
0,e47729,2024-02-01 00:02:20+00:00,OEIRAS,[motorized_mobility_708112],NaN,NaN
1,e47730,2024-02-01 00:04:06+00:00,LISBOA,[motorized_mobility_708112],NaN,NaN
2,e21055,2024-02-01 00:05:52+00:00,OEIRAS,NaN,[fast_mobility_286761],NaN
3,e21056,2024-02-01 00:06:27+00:00,OEIRAS,NaN,[fast_mobility_286761],NaN
4,e37167,2024-02-01 00:23:45+00:00,OEIRAS,[motorized_mobility_543412],NaN,NaN
...,...,...,...,...,...,...
70809,e18976,2024-03-13 23:22:26+00:00,OEIRAS,[motorized_mobility_257543],NaN,NaN
70810,e38269,2024-03-13 23:23:10+00:00,OEIRAS,NaN,NaN,[slow_mobility_559212]
70811,e38270,2024-03-13 23:25:03+00:00,CASCAIS,NaN,NaN,[slow_mobility_559212]
70812,e34727,2024-03-13 23:50:30+00:00,OEIRAS,[motorized_mobility_505772],NaN,NaN


In [30]:
ocel_df_oeiras_with_mode.get_extended_table()

,ocel:eid,ocel:timestamp,ocel:activity,ocel:type:bicycle,ocel:type:train_or_metro,ocel:type:foot,ocel:type:car,ocel:type:long_distance_train,ocel:type:bus
0,e8183,2024-02-01 06:18:21+00:00,OEIRAS,[bicycle_413735],NaN,NaN,NaN,NaN,NaN
1,e8184,2024-02-01 06:23:12+00:00,OEIRAS,[bicycle_413735],NaN,NaN,NaN,NaN,NaN
2,e12319,2024-02-01 07:53:57+00:00,OEIRAS,NaN,[train_or_metro_645310],NaN,NaN,NaN,NaN
3,e18133,2024-02-01 07:56:58+00:00,OEIRAS,[bicycle_946707],NaN,NaN,NaN,NaN,NaN
4,e12320,2024-02-01 08:00:08+00:00,CASCAIS,NaN,[train_or_metro_645310],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
19801,e15514,2024-03-13 22:31:24+00:00,CASCAIS,NaN,NaN,[foot_813027],NaN,NaN,NaN
19802,e2307,2024-03-13 23:12:19+00:00,OEIRAS,[bicycle_125026],NaN,NaN,NaN,NaN,NaN
19803,e2308,2024-03-13 23:16:18+00:00,SINTRA,[bicycle_125026],NaN,NaN,NaN,NaN,NaN
19804,e5303,2024-03-13 23:18:18+00:00,OEIRAS,[bicycle_257543],NaN,NaN,NaN,NaN,NaN


#### 8. Discover Object-centric Directly-Followed Graph 

In [28]:
ocdfg_oeiras = pm4py.discover_ocdfg(ocel_df_oeiras)
pm4py.view_ocdfg(ocdfg_oeiras annotation='frequency', format='html')

In [27]:
ocdfg_oeiras_with_mode = pm4py.discover_ocdfg(ocel_df_oeiras_with_mode)
pm4py.view_ocdfg(ocdfg_oeiras_with_mode, annotation='frequency', format='html')